In [1]:
import numpy as np
import pandas as pd

# データ集約とグループ演算

In [4]:
df=pd.DataFrame({'key1':['a','a','b','b','a'],
                 'key2':['one','two','one','two','one'],
                 'data1':np.random.randn(5),
                 'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.490232,-0.123229
1,a,two,-0.082486,-0.361932
2,b,one,-0.680382,0.858691
3,b,two,1.011444,0.365075
4,a,one,-0.849499,1.680805


In [5]:
grouped=df['data1'].groupby(df['key1'])
grouped

In [6]:
grouped.mean()

key1
a   -0.474072
b    0.165531
Name: data1, dtype: float64

In [7]:
means=df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     one    -0.669865
      two    -0.082486
b     one    -0.680382
      two     1.011444
Name: data1, dtype: float64

In [8]:
means.unstack()

key2,one,two
key1,,
a,-0.669865,-0.082486
b,-0.680382,1.011444


In [9]:
states=np.array(['ohio','california','california','ohio','ohio'])
years=np.array([2005,2005,2006,2005,2006])
df['data1'].groupby([states,years]).mean()

california  2005   -0.082486
            2006   -0.680382
ohio        2005    0.260606
            2006   -0.849499
Name: data1, dtype: float64

In [10]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.474072,0.398548
b,0.165531,0.611883


In [11]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.669865  0.778788
     two  -0.082486 -0.361932
b    one  -0.680382  0.858691
     two   1.011444  0.365075

In [12]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## グループを繰り返す

In [13]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.490232 -0.123229
1    a  two -0.082486 -0.361932
4    a  one -0.849499  1.680805
b
  key1 key2     data1     data2
2    b  one -0.680382  0.858691
3    b  two  1.011444  0.365075


In [14]:
for (k1,k2), group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.490232 -0.123229
4    a  one -0.849499  1.680805
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.082486 -0.361932
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.680382  0.858691
('b', 'two')
  key1 key2     data1     data2
3    b  two  1.011444  0.365075


In [16]:
pieces=dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.680382,0.858691
3,b,two,1.011444,0.365075


In [17]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [18]:
grouped=df.groupby(df.dtypes,axis=1)
for dtype,group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.490232 -0.123229
1 -0.082486 -0.361932
2 -0.680382  0.858691
3  1.011444  0.365075
4 -0.849499  1.680805
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 列または列のサブセットを選択

In [19]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.778788
     two  -0.361932
b    one   0.858691
     two   0.365075

In [20]:
s_grouped=df.groupby(['key1','key2'])['data2']
s_grouped

In [21]:
s_grouped.mean()

key1  key2
a     one     0.778788
      two    -0.361932
b     one     0.858691
      two     0.365075
Name: data2, dtype: float64

### グループを作る

In [22]:
people=pd.DataFrame(np.random.randn(5,5),
                   columns=['a','b','c','d','e'],
                   index=['joe','steve','wes','jim','travis'])
people.iloc[2:3,[1,2]]=np.nan
people

,a,b,c,d,e
joe,-0.152097,-0.188426,0.884480,1.524735,-0.201549
steve,-0.494641,0.869917,0.558046,0.177415,-1.357415
wes,-0.187756,NaN,NaN,0.474138,-0.330880
jim,-0.270620,-0.584764,-0.870098,1.224880,-0.515039
travis,0.815894,0.446888,1.179248,0.161730,-0.579151


In [23]:
mapping={'a':'red','b':'red','c':'blue','d':'blue','e':'red','f':'orange'}
by_column=people.groupby(mapping,axis=1)
by_column.sum()


,blue,red
joe,2.409215,-0.542072
steve,0.735461,-0.982139
wes,0.474138,-0.518635
jim,0.354782,-1.370423
travis,1.340977,0.683630


In [25]:
map_series=pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [26]:
people.groupby(map_series,axis=1).count()

,blue,red
joe,2,3
steve,2,3
wes,1,2
jim,2,3
travis,2,3


### 関数でグループを作る

In [27]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.610473,-0.773191,0.014382,3.223753,-1.047467
5,-0.494641,0.869917,0.558046,0.177415,-1.357415
6,0.815894,0.446888,1.179248,0.161730,-0.579151


In [28]:
key_list=['one','one','one','two','two']
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one -0.187756 -0.188426  0.884480  0.474138 -0.330880
  two -0.270620 -0.584764 -0.870098  1.224880 -0.515039
5 one -0.494641  0.869917  0.558046  0.177415 -1.357415
6 two  0.815894  0.446888  1.179248  0.161730 -0.579151

In [29]:
columns=pd.MultiIndex.from_arrays([['us','us','us','jp','jp'],
                                  [1,3,4,1,3]],names=['cty','tenor'])
hier_df=pd.DataFrame(np.random.randn(4,5),columns=columns)
hier_df

cty          us                            jp          
tenor         1         3         4         1         3
0     -0.038296 -1.570423 -0.506780 -0.219772  0.488276
1     -1.380167 -0.112962 -0.179617 -0.379276 -0.525317
2     -0.471181 -1.312754 -0.363196  0.875353 -1.247707
3     -0.704059 -0.801923  0.188805 -0.325945  0.755927

In [30]:
hier_df.groupby(level='cty',axis=1).count()

cty,jp,us
0,2,3
1,2,3
2,2,3
3,2,3


## データ集合